# TP - HMMA238

Par Mathias Gout et Julie Røste.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
#plt.rcParams["figure.figsize"] = (17,8)

## Exercice 1 - Le jeu de la vie

### Implémentation sans `numpy`

### Question 4:

In [ ]:
from utils import calcul_nb_voisins
from utils import iteration_jeu

Z_init = [[0,0,0,0,0,0], 
          [0,0,0,1,0,0], 
          [0,1,0,1,0,0], 
          [0,0,1,1,0,0],
          [0,0,0,0,0,0], 
          [0,0,0,0,0,0]]
     
calcul_nb_voisins(Z_init)

`calcul_nb_voisins(Z)` affiche le nombre de voisins vivants de chaque cellules

### Question 5:

In [ ]:
# On rappelle que l'on a renommé Z par Z_init
Z_init

In [ ]:
from utils import plotJeuDeLaVie

plotJeuDeLaVie(10, np.asarray(Z_init), iteration_jeu)

### Question 6:

On remarque que les cellules vivantes de l'itération 0 sont disposées de la même façon que celles de l'itération 4. 
<br/>Entre l'itération 0 et 4, les cellules vivantes se sont toutes déplacées d'une cellule en bas et d'une cellule à droite.

A partir de l'itération 7, les cellules ne changent plus d'état. 
Les quatres cellules vivantes sont regroupées en bas à droite et ont toutes 3 voisins vivants (aucun décès possible).
<br/>Comme aucune cellule morte n'a 3 voisins vivants (pas de naissance possible), il n'y aura pas d'évolution dans les prochaines itérations.


### Implémentation avec `numba`

### Question 7:

In [ ]:
from numba import jit

Les fonctions utilisant `numpy` sont `calcul_np_voisins_np` et `iteration_jeu_np`. 

In [ ]:
# Rappel
Z_init = [[0,0,0,0,0,0],
          [0,0,0,1,0,0], 
          [0,1,0,1,0,0], 
          [0,0,1,1,0,0],
          [0,0,0,0,0,0], 
          [0,0,0,0,0,0]]

# Pour pouvoir l'utiliser dans calcul_nb_voisins_np et iteration_jeu_np
Z_np = np.array(Z_init)

In [ ]:
from utils import calcul_nb_voisins_np, iteration_jeu_np

On commence par montrer que nos fonctions `calcul_nb_voisins_np` et `iteration_jeu_np` fonctionnent correctement.

In [ ]:
print(calcul_nb_voisins_np(Z_np))
iteration_jeu_np(Z_np)

On obtient bien les résultats attendus.

On calcule maintenant les temps de calculs de `calcul_nb_voisins`, `iteration_jeu` (sans `numba`) et `calcul_nb_voisins_nb`, `iteration_jeu_np` (avec `numba`).
<br/> On les comparera et on déterminera dans quel cas on obtient le meilleur résultat.

In [ ]:
%timeit calcul_nb_voisins(Z_init)
%timeit iteration_jeu(Z_init)

In [ ]:
%timeit calcul_nb_voisins_np(Z_np)
%timeit iteration_jeu_np(Z_np)

Les temps de calculs quand on utilise `numba` sont bien plus réduit.

### Question 8:

In [ ]:
from ipywidgets import interact, fixed
from utils import iteration_jeu_np, plotJeuDeLaVie_manySP

In [ ]:
n = 20

Z_huge = np.zeros((n,n))
Z_np = np.array( 
    [[0, 0, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 1, 0, 1, 0, 0],
     [0, 0, 1, 1, 0, 0],
     [0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0]])

Z_huge[10:16,10:16] = Z_np

interact(plotJeuDeLaVie_manySP, nbIterations=(0, 30, 1), Z=fixed(Z_huge), iter_func=fixed(iteration_jeu_np));

Avec $n=20$, la matrice `Z_huge` converge à partir de la $23^{ème}$ itération. 

## Exercice 2 - Régression logistique

### Question 1:

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

In [ ]:
X = mnist.data.astype('float64') 
y = mnist.target.astype('float64')

### Question 2:

In [ ]:
# On garde que les 3 et 7
X = X[(y==3) + (y==7)] 
y = y[(y==3) + (y==7)] 

# Premier individu de la classe 3 et 7
premier_3 = np.where(y==3)[0][0]
premier_7 = np.where(y==7)[0][0]

# On affiche les 2 individus
plt.subplot(1, 2, 1)
plt.title("1er ind. de la classe 3")
plt.imshow(X[premier_3].reshape(28,28))
plt.subplot(1, 2, 2)
plt.title("1er ind. de la classe 7")
plt.imshow(X[premier_7].reshape(28,28))

### Question 3:

In [ ]:
from sklearn.linear_model import LogisticRegression 

# Regression logistique 
reg_log = LogisticRegression(fit_intercept=False, solver='liblinear').fit(X, y)

### Question 4:

In [ ]:
from ipywidgets import interact, fixed
from utils import fig_digit 

In [ ]:
# Nous avons fait varier alpha de 1 en 1 pour plus de fluidité
interact(fig_digit, x=fixed(X[premier_7]), w=fixed(reg_log.coef_), alpha=(0,100,1));

### Question 5:

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(xlim=(0, 27), ylim=(27, 0))
ims = []

for i in range(100):
    im = fig_digit(X[premier_7], reg_log.coef_, i)
    ims.append([im])
    
anim = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

anim.to_html5_video()
plt.show()

### Question 6: 

D'equation (1) en exerice 4, on peut écrire $\alpha$ comme: 
$$\alpha  = (x-x_{mod}) {||w||}^2 (w^Tx)^{-1}$$

C'est comme la taux d'apprentissage de l'algorithme 